# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234011773789                   -0.50    9.0         
  2   -7.250263580936       -1.79       -1.40    1.0   5.36ms
  3   -7.251178714544       -3.04       -2.19    1.0   5.40ms
  4   -7.251307547350       -3.89       -2.27    2.0   6.81ms
  5   -7.251335859566       -4.55       -2.83    1.0   5.42ms
  6   -7.251336727083       -6.06       -2.97    2.0   6.43ms
  7   -7.251338597670       -5.73       -3.46    1.0   5.55ms
  8   -7.251338732180       -6.87       -3.68    2.0   7.51ms
  9   -7.251338792743       -7.22       -4.18    1.0   6.69ms
 10   -7.251338798612       -8.23       -4.95    3.0   8.52ms
 11   -7.251338798691      -10.10       -5.44    1.0   7.32ms
 12   -7.251338798698      -11.16       -5.68    1.0   8.84ms
 13   -7.251338798704      -11.24       -6.41    1.0   8.76ms
 14   -7.251338798704      -12.42       -6.48    3.0   63.7ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.037263369155274166
[ Info: Arnoldi iteration step 2: normres = 0.5044378256234491
[ Info: Arnoldi iteration step 3: normres = 0.7771165801489451
[ Info: Arnoldi iteration step 4: normres = 0.4561941583474627
[ Info: Arnoldi iteration step 5: normres = 0.4800884284542249
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (9.46e-02, 7.74e-02, 4.07e-01, 2.23e-01, 1.69e-02)
[ Info: Arnoldi iteration step 6: normres = 0.38538812884099394
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (3.11e-02, 1.89e-01, 2.85e-01, 1.41e-01, 9.55e-02)
[ Info: Arnoldi iteration step 7: normres = 0.0789089417364965
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.25e-03, 1.79e-02, 1.56e-02, 5.09e-02, 4.45e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11862330539505117
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.46e-